In [1]:
import k3d
import numpy as np

import sys
sys.path.insert(0, "..")

from TimingCalib.MPMT import MPMT
from TimingCalib.PMT import PMT
from TimingCalib.LED import LED
from TimingCalib.WCD import WCD

from scipy.spatial.transform import Rotation as R

In [2]:
wcte = WCD(None, 'WCTE', {}, {})

In [11]:
# Produce a light pulse from an LED for visualization

# choose leds to pulse
mpmt_emitter = wcte.mpmts[54]
led_emitters = [mpmt_emitter.leds[i] for i in range(3)]

pattern = 'ring'
if pattern == 'ring':
    # to produce light in a ring at 15-20 degrees, change the properties of that led
    cone_angle = 17.5/180.*np.pi
    cone_ring_width = 2.5/180.*np.pi
    for led_emitter in led_emitters:
        led_emitter.prop_true['cone_angle'] = cone_angle
        led_emitter.prop_true['cone_ring_width'] = cone_ring_width
if pattern == 'cone':
    cone_angle = 20./180.*np.pi
    cone_ring_width = 0.
    for led_emitter in led_emitters:
        led_emitter.prop_true['cone_angle'] = cone_angle
        led_emitter.prop_true['cone_ring_width'] = cone_ring_width

# produce an event from the light pulse
led_volts = 8.
wcte_events = []
for led_emitter in led_emitters:
    wcte_events.append(wcte.pulse_LED(led_emitter, led_volts))

In [12]:
# Show all mPMTs and some PMTs+LEDs in the WCTE detector: just the PMTs hit and the emitting LED

led_colors = [0xff0000,0x00ff00,0xffff00]
color_pmt_hit = 0x00ff00

plot = k3d.plot()

# mpmt coordinates
color_z = 0x0000ff
color_x = 0xff0000
color_y = 0x00ff00
origins = []
z_vecs = []
x_vecs = []
vec_length = 100. # length of x,z vectors to show for each pmt

color_mpmt = 0xabb2b9
n_point_mpmt = 8
indices_mpmt = []
for i in range(n_point_mpmt):
    indices_mpmt.append([i,(i+1)%n_point_mpmt])
    
n_point_ft = 20
indices_ft = []
for i in range(n_point_ft):
    indices_ft.append([i,(i+1)%n_point_ft])
    
# pmt faces:
color_pmt = 0xdc7633
n_point_pmt = 20
indices_pmt = []
for i in range(n_point_pmt):
    indices_pmt.append([i,(i+1)%n_point_pmt])
    
# led coordinates
led_origins = []
led_z_vecs = []
led_vec_length = 30. # length of z vectors to show for each led

color_led = 0x00ff00
    
# draw the extent of the mpmt baseplates
for i_mpmt,mpmt in enumerate(wcte.mpmts):
    
    location, direction_x, direction_z = mpmt.get_orientation('design')
    # lists to show mPMT coordinate systems
    z_vec = list(np.array(direction_z)*vec_length)
    x_vec = list(np.array(direction_x)*vec_length)
    origins.append(location)
    z_vecs.append(z_vec)
    x_vecs.append(x_vec)
    
    baseplate_points = np.array(mpmt.get_xy_points('design'),dtype=np.float32)
    plt_baseplate = k3d.lines(baseplate_points, indices_mpmt, indices_type='segment', color=color_mpmt)
    plot += plt_baseplate
    
    feedthrough_points = np.array(mpmt.get_xy_points('design', feature='feedthrough'),dtype=np.float32)
    plt_feedthrough = k3d.lines(feedthrough_points, indices_ft, indices_type='segment', color=color_mpmt)
    plot += plt_feedthrough
    
    # show the hit pmts
    for i_pmt, pmt in enumerate(mpmt.pmts):
        for i_led,wcte_event in enumerate(wcte_events):
            if wcte_event.pmt_signals[i_mpmt][i_pmt] is not None:

                # k3d expects ndarray of float32 for segments
                circle_points = np.array(pmt.get_circle_points(n_point_pmt,'design'),dtype=np.float32)
                plt_circle = k3d.lines(circle_points, indices_pmt, indices_type='segment', color=led_colors[i_led])
                plot += plt_circle


for i_led,led_emitter in enumerate(led_emitters):
    location, direction_x, direction_z = led_emitter.get_orientation('design')
    z_vec = list(np.array(direction_z)*led_vec_length)
    led_origins.append(location)
    led_z_vecs.append(z_vec)

#plot led locations and axis (different colors)
for i_led,led_emitter in enumerate(led_emitters):    
    led_locations = np.array([led_origins[i_led]], dtype=np.float32)    
    plt_leds = k3d.points(positions=led_locations,
                        point_size=8.,
                        shader='3d',
                        color=led_colors[i_led])
    plot += plt_leds

    plt_led_z_vecs = k3d.vectors(origins=[led_origins[i_led]], vectors=[led_z_vecs[i_led]], color=color_z, head_size=50.)
    plot += plt_led_z_vecs

plot.display()

Output()